In [5]:
import requests
import pandas as pd

# Endpoint apenas com os campos necessários
url = "https://restcountries.com/v3.1/all?fields=name,cca2,cca3,ccn3,cioc"

response = requests.get(url)

if response.status_code == 200:
    print("Conexão bem-sucedida com a API REST Countries!")

    paises = response.json()
    lista_paises = []

    for pais in paises:
        nome = pais.get('name', {}).get('common', 'Não informado')
        official_name = pais.get('name', {}).get('official', 'Não informado')
        codigo_alpha2 = pais.get('cca2', 'Não informado')
        codigo_alpha3 = pais.get('cca3', 'Não informado')
        codigo_numerico = pais.get('ccn3', 'Não informado')
        codigo_olimpico = pais.get('cioc', 'Não informado')

        lista_paises.append({
            'nome': nome,
            'official_name': official_name,
            'codigo_alpha2': codigo_alpha2,
            'codigo_alpha3': codigo_alpha3,
            'codigo_numerico': codigo_numerico,
            'codigo_olimpico': codigo_olimpico
        })

    df_lista_paises = pd.DataFrame(lista_paises)
    print("DataFrame criado com sucesso!")
    display(df_lista_paises)

else:
    print(f"Erro na requisição. Código de status: {response.status_code}")
    print("Resposta do servidor:", response.text[:200])


Conexão bem-sucedida com a API REST Countries!
DataFrame criado com sucesso!


,nome,official_name,codigo_alpha2,codigo_alpha3,codigo_numerico,codigo_olimpico
0,Jamaica,Jamaica,JM,JAM,388,JAM
1,Comoros,Union of the Comoros,KM,COM,174,COM
2,Zimbabwe,Republic of Zimbabwe,ZW,ZWE,716,ZIM
3,Sint Maarten,Sint Maarten,SX,SXM,534,
4,Laos,Lao People's Democratic Republic,LA,LAO,418,LAO
...,...,...,...,...,...,...
245,Austria,Republic of Austria,AT,AUT,040,AUT
246,Netherlands,Kingdom of the Netherlands,NL,NLD,528,NED
247,Svalbard and Jan Mayen,Svalbard og Jan Mayen,SJ,SJM,744,
248,Singapore,Republic of Singapore,SG,SGP,702,SGP


In [6]:
from datetime import datetime
import pytz

fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

df_lista_paises['data_carga'] = agora_brasil.strftime('%Y-%m-%d %H:%M:%S')


In [7]:
#!pip install pandasql
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [8]:
consultaSQL = """

SELECT *
FROM df_lista_paises 
--where nome = "Brazil"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,nome,official_name,codigo_alpha2,codigo_alpha3,codigo_numerico,codigo_olimpico,data_carga
0,Jamaica,Jamaica,JM,JAM,388,JAM,2025-09-24 13:08:26
1,Comoros,Union of the Comoros,KM,COM,174,COM,2025-09-24 13:08:26
2,Zimbabwe,Republic of Zimbabwe,ZW,ZWE,716,ZIM,2025-09-24 13:08:26
3,Sint Maarten,Sint Maarten,SX,SXM,534,,2025-09-24 13:08:26
4,Laos,Lao People's Democratic Republic,LA,LAO,418,LAO,2025-09-24 13:08:26
...,...,...,...,...,...,...,...
245,Austria,Republic of Austria,AT,AUT,040,AUT,2025-09-24 13:08:26
246,Netherlands,Kingdom of the Netherlands,NL,NLD,528,NED,2025-09-24 13:08:26
247,Svalbard and Jan Mayen,Svalbard og Jan Mayen,SJ,SJM,744,,2025-09-24 13:08:26
248,Singapore,Republic of Singapore,SG,SGP,702,SGP,2025-09-24 13:08:26


In [9]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_pais.csv", index=False)


In [10]:
# ==========================================================
# Importando bibliotecas necessárias
# ==========================================================
from sqlalchemy import create_engine

# ==========================================================
# Definindo as credenciais do banco (Render)
# ==========================================================
host = "dpg-d39eabemcj7s73f0vcv0-a.oregon-postgres.render.com"
port = "5432"
database = "projeto_paises"
user = "user"
password = "QP7z5ILeinmGr76fDpSeNIiqLOeXL4WE"

# ==========================================================
# Criando a conexão com o PostgreSQL usando SQLAlchemy
# ==========================================================
engine = create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/{database}"
)


# ==========================================================
# Salvando no banco (tabela: paises)
# if_exists:
#   - 'replace' -> apaga e recria a tabela
#   - 'append' -> adiciona os dados
# ==========================================================
df_export.to_sql("tbl_dim_pais", engine, if_exists="replace", index=False)

print("✅ Dados salvos no banco Render com sucesso!")


✅ Dados salvos no banco Render com sucesso!


In [11]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
cur.execute("DELETE FROM TBL_DIM_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_DIM_PAIS (   
                 nome,
                 codigo_alpha2,
                 codigo_alpha3,
                 codigo_numerico,
                 codigo_olimpico,
                 official_name,
                 data_carga)
                 VALUES (%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["nome"],
        colunas_df["codigo_alpha2"],
        colunas_df["codigo_alpha3"],
        colunas_df["codigo_numerico"],
        colunas_df["codigo_olimpico"],
        colunas_df["official_name"],
        colunas_df["data_carga"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()